In [ ]:
import os
from diagrams import Diagram, Cluster
from diagrams.custom import Custom
from diagrams.onprem.container import Docker
from diagrams.onprem.database import PostgreSQL
from diagrams.onprem.inmemory import Redis
from diagrams.programming.framework import FastAPI

project_dir = os.getcwd()

with Diagram("Docker Compose Architecture", show=True, direction="TB", outformat="png", filename="docker_architecture"):
    # Airflow Services Cluster
    with Cluster("🚀 Airflow Services"):
        webserver = Docker("🖥️ airflow-webserver")
        scheduler = Docker("📅 airflow-scheduler")
        worker = Docker("⚙️ airflow-worker")
        flower = Docker("📊 flower")
        init = Docker("▶️ airflow-init")

        # New Docker-in-Docker Training Container
        docker_training = Docker("🔨 DockerOperator\nModel Training")
        
        # Connect DockerOperator to scheduler and worker
        scheduler >> docker_training
        worker >> docker_training

    # Airflow UI Services
    with Cluster("💾 Airflow-UI"):
        airflow_ui = Custom("📈 airflow_ui", os.path.join(project_dir, "z_images_README" "Airflow_UI.png"))
        webserver >> airflow_ui
        airflow_ui >> webserver

    # Database Services Cluster
    with Cluster("💾 Database Services"):
        postgres = PostgreSQL("🐘 postgres")
        mlflow_postgres = PostgreSQL("🐘 mlflow-postgres")
        redis = Redis("🔄 redis")

 # FastAPI App Cluster with Swagger UI    
    with Cluster("FastAPI App"):
        swagger = Custom("📝 Swagger UI", os.path.join(project_dir, "z_images_README", "Swagger_UI.png"))
        fastapi = Custom("⚡ fastapi-app", os.path.join(project_dir, "z_images_README", "FastAPI_UI.png"))

    
    # ML Services Cluster
    with Cluster("🧠 ML Services"):
        mlflow = Custom("📈 mlflow", os.path.join(project_dir, "z_images_README", "MLflow.png"))
        mlflow_ui = Custom("📈 mlflow", os.path.join(project_dir, "z_images_README", "MLflow_UI.png"))
    
   
    # Define dependencies
    # Airflow services depend on Database services
    webserver >> postgres
    webserver >> redis
    scheduler >> postgres
    scheduler >> redis
    worker >> postgres
    worker >> redis
    flower >> postgres
    flower >> redis
    init >> postgres
    init >> redis

    # Docker-in-Docker Training Dependencies
    docker_training >> fastapi  # Training uses FastAPI container
    docker_training >> mlflow   # Logs and tracks to MLflow
    docker_training >> redis
    docker_training >> postgres

    # ML service dependencies
    mlflow >> mlflow_postgres
    fastapi >> mlflow
    mlflow >> fastapi